In [1]:
!pip install tensorflow
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install dask
import dask.dataframe as dd

In [4]:
root_dir = "/content/drive/MyDrive/CS 297/mimic-iv-2.2"

In [5]:
admissionsPath = root_dir+"/hosp/admissions.csv"
admissionsData = dd.read_csv(admissionsPath, assume_missing=True)

In [6]:
patientsPath = root_dir + "/hosp/patients.csv"
patientsData = dd.read_csv(patientsPath)

In [7]:
prescriptionsPath = root_dir + "/hosp/prescriptions.csv"
prescriptionsData = dd.read_csv(prescriptionsPath)

In [8]:
procedureEventsPath = root_dir + "/icu/procedureevents.csv/procedureevents.csv"
procedureEventsData = dd.read_csv(procedureEventsPath)

In [9]:
inputEventsPath = root_dir + "/icu/inputevents.csv/inputevents.csv"
inputEventsData = dd.read_csv(inputEventsPath, assume_missing=True)

In [10]:
inputEventsData.head()

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,amount,amountuom,...,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10000032.0,29079034.0,39553978.0,66056.0,2180-07-23 21:10:00,2180-07-23 21:11:00,2180-07-23 21:10:00,226452.0,100.000000,ml,...,Main order parameter,Bolus,39.4,100.0,ml,0.0,0.0,FinishedRunning,100.0,100.0
1,10000032.0,29079034.0,39553978.0,88981.0,2180-07-23 17:00:00,2180-07-23 17:01:00,2180-07-23 18:56:00,226452.0,200.000000,ml,...,Main order parameter,Bolus,39.4,200.0,ml,0.0,0.0,FinishedRunning,200.0,200.0
2,10000032.0,29079034.0,39553978.0,88981.0,2180-07-23 17:00:00,2180-07-23 17:30:00,2180-07-23 17:02:00,220862.0,49.999999,ml,...,Main order parameter,Continuous IV,39.4,50.0,ml,0.0,0.0,FinishedRunning,50.0,100.0
3,10000032.0,29079034.0,39553978.0,88981.0,2180-07-23 17:33:00,2180-07-23 18:03:00,2180-07-23 18:16:00,220862.0,49.999999,ml,...,Main order parameter,Continuous IV,39.4,50.0,ml,0.0,0.0,FinishedRunning,50.0,100.0
4,10000032.0,29079034.0,39553978.0,88981.0,2180-07-23 18:56:00,2180-07-23 18:57:00,2180-07-23 18:56:00,226452.0,100.000000,ml,...,Main order parameter,Bolus,39.4,100.0,ml,0.0,0.0,FinishedRunning,100.0,100.0


In [11]:
d_itemsPath = root_dir + "/icu/d_items.csv/d_items.csv"
d_itemsData = dd.read_csv(d_itemsPath)

In [12]:
d_itemsData.head()

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,NaN,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN


In [13]:
chartEventsPath = root_dir + "/icu/chartevents.csv/chartevents.csv"
dtype = {"value":"object"}
chartEventsData = dd.read_csv(chartEventsPath, assume_missing=True, dtype = dtype)

In [14]:
chartEventsData.head()

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032.0,29079034.0,39553978.0,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220179.0,82,82.0,mmHg,0.0
1,10000032.0,29079034.0,39553978.0,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220180.0,59,59.0,mmHg,0.0
2,10000032.0,29079034.0,39553978.0,47007.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220181.0,63,63.0,mmHg,0.0
3,10000032.0,29079034.0,39553978.0,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220045.0,94,94.0,bpm,0.0
4,10000032.0,29079034.0,39553978.0,47007.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220179.0,85,85.0,mmHg,0.0


In [15]:
SEED = 50

In [16]:
chartEventsData.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 11 entries, subject_id to warning
dtypes: object(4), float64(7)

In [17]:
d_itemsData.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 9 entries, itemid to highnormalvalue
dtypes: object(6), float64(2), int64(1)

In [18]:
d_itemsData = d_itemsData.drop(columns=['abbreviation', 'linksto', 'unitname', 'param_type', 'lownormalvalue', 'highnormalvalue'])

In [19]:
d_itemsData.head()

,itemid,label,category
0,220001,Problem List,General
1,220003,ICU Admission date,ADT
2,220045,Heart Rate,Routine Vital Signs
3,220046,Heart rate Alarm - High,Alarms
4,220047,Heart Rate Alarm - Low,Alarms


In [20]:
inputEventsData.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 26 entries, subject_id to originalrate
dtypes: object(11), float64(15)

In [21]:
inputEventsData = inputEventsData[['subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 'storetime', 'itemid', 'amount', 'amountuom', 'rate', 'rateuom', 'ordercategoryname', 'secondaryordercategoryname']]

In [22]:
inputEventsData.head()

,subject_id,hadm_id,stay_id,starttime,endtime,storetime,itemid,amount,amountuom,rate,rateuom,ordercategoryname,secondaryordercategoryname
0,10000032.0,29079034.0,39553978.0,2180-07-23 21:10:00,2180-07-23 21:11:00,2180-07-23 21:10:00,226452.0,100.000000,ml,NaN,NaN,14-Oral/Gastric Intake,NaN
1,10000032.0,29079034.0,39553978.0,2180-07-23 17:00:00,2180-07-23 17:01:00,2180-07-23 18:56:00,226452.0,200.000000,ml,NaN,NaN,14-Oral/Gastric Intake,NaN
2,10000032.0,29079034.0,39553978.0,2180-07-23 17:00:00,2180-07-23 17:30:00,2180-07-23 17:02:00,220862.0,49.999999,ml,100.0,mL/hour,04-Fluids (Colloids),NaN
3,10000032.0,29079034.0,39553978.0,2180-07-23 17:33:00,2180-07-23 18:03:00,2180-07-23 18:16:00,220862.0,49.999999,ml,100.0,mL/hour,04-Fluids (Colloids),NaN
4,10000032.0,29079034.0,39553978.0,2180-07-23 18:56:00,2180-07-23 18:57:00,2180-07-23 18:56:00,226452.0,100.000000,ml,NaN,NaN,14-Oral/Gastric Intake,NaN


In [23]:
# Perform the join based on the common field
updatedInputEventsData = dd.merge(d_itemsData, inputEventsData, on=['itemid'], how='inner')

In [24]:
updatedInputEventsData.head()

,itemid,label,category,subject_id,hadm_id,stay_id,starttime,endtime,storetime,amount,amountuom,rate,rateuom,ordercategoryname,secondaryordercategoryname
0,220862,Albumin 25%,Blood Products/Colloids,10000032.0,29079034.0,39553978.0,2180-07-23 17:00:00,2180-07-23 17:30:00,2180-07-23 17:02:00,49.999999,ml,100.0,mL/hour,04-Fluids (Colloids),NaN
1,220862,Albumin 25%,Blood Products/Colloids,10000032.0,29079034.0,39553978.0,2180-07-23 17:33:00,2180-07-23 18:03:00,2180-07-23 18:16:00,49.999999,ml,100.0,mL/hour,04-Fluids (Colloids),NaN
2,220862,Albumin 25%,Blood Products/Colloids,10002428.0,28662225.0,38875437.0,2156-04-21 11:48:00,2156-04-21 14:18:00,2156-04-21 11:48:00,499.999988,ml,200.0,mL/hour,04-Fluids (Colloids),NaN
3,220862,Albumin 25%,Blood Products/Colloids,10002428.0,28662225.0,38875437.0,2156-04-23 02:07:00,2156-04-23 02:37:00,2156-04-23 02:07:00,49.999999,ml,100.0,mL/hour,04-Fluids (Colloids),NaN
4,220862,Albumin 25%,Blood Products/Colloids,10002428.0,28662225.0,38875437.0,2156-04-23 02:30:00,2156-04-23 03:00:00,2156-04-23 05:19:00,49.999999,ml,100.0,mL/hour,04-Fluids (Colloids),NaN


In [25]:
updatedInputEventsData['itemid'].unique().compute()

0      220862
1      220864
2      220949
3      220950
4      220952
        ...  
319    227976
320    229861
321    226024
322    228361
323    227090
Name: itemid, Length: 324, dtype: int64

In [26]:
chartdtypes = chartEventsData.dtypes
print(chartdtypes.to_dict())

{'subject_id': dtype('float64'), 'hadm_id': dtype('float64'), 'stay_id': dtype('float64'), 'caregiver_id': dtype('float64'), 'charttime': dtype('O'), 'storetime': dtype('O'), 'itemid': dtype('float64'), 'value': dtype('O'), 'valuenum': dtype('float64'), 'valueuom': dtype('O'), 'warning': dtype('float64')}


In [27]:
chartEventsData = chartEventsData.drop(columns=['caregiver_id'])

In [28]:
chartEventsData.head()

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10000032.0,29079034.0,39553978.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220179.0,82,82.0,mmHg,0.0
1,10000032.0,29079034.0,39553978.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220180.0,59,59.0,mmHg,0.0
2,10000032.0,29079034.0,39553978.0,2180-07-23 21:01:00,2180-07-23 22:15:00,220181.0,63,63.0,mmHg,0.0
3,10000032.0,29079034.0,39553978.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220045.0,94,94.0,bpm,0.0
4,10000032.0,29079034.0,39553978.0,2180-07-23 22:00:00,2180-07-23 22:15:00,220179.0,85,85.0,mmHg,0.0


In [29]:
# Perform the join based on the common field
updatedChartEventsData = dd.merge(d_itemsData, chartEventsData, on=['itemid'], how='inner')

In [30]:
updatedChartEventsData.head()

,itemid,label,category,subject_id,hadm_id,stay_id,charttime,storetime,value,valuenum,valueuom,warning
0,220001,Problem List,General,10000980.0,26913865.0,39765666.0,2189-06-27 15:24:57,NaN,.Care Plan - Altered Respiratory Function: Ine...,NaN,NaN,NaN
1,220001,Problem List,General,10001217.0,24597018.0,37067082.0,2157-11-21 07:26:41,NaN,.Care Plan - Post-operative,NaN,NaN,NaN
2,220001,Problem List,General,10001217.0,27703517.0,34592300.0,2157-12-19 21:56:58,NaN,.Care Plan - Post-operative,NaN,NaN,NaN
3,220001,Problem List,General,10001725.0,25563031.0,31205490.0,2110-04-11 18:56:18,NaN,.Care Plan - Post-operative,NaN,NaN,NaN
4,220001,Problem List,General,10001725.0,25563031.0,31205490.0,2110-04-11 20:39:56,NaN,.Care Plan - Altered Respiratory Function: Ine...,NaN,NaN,NaN


In [31]:
updatedInputEventsData.dtypes.to_dict()

{'itemid': dtype('int64'),
 'label': dtype('O'),
 'category': dtype('O'),
 'subject_id': dtype('float64'),
 'hadm_id': dtype('float64'),
 'stay_id': dtype('float64'),
 'starttime': dtype('O'),
 'endtime': dtype('O'),
 'storetime': dtype('O'),
 'amount': dtype('float64'),
 'amountuom': dtype('O'),
 'rate': dtype('float64'),
 'rateuom': dtype('O'),
 'ordercategoryname': dtype('O'),
 'secondaryordercategoryname': dtype('O')}

In [32]:
updatedChartEventsData.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 12 entries, itemid to warning
dtypes: object(6), float64(5), int64(1)

In [33]:
updatedChartEventsData.npartitions

471

In [34]:
#updatedChartEventsData = updatedChartEventsData.sample(frac=0.1)

In [35]:
#updatedChartEventsData = updatedChartEventsData.repartition(1000)

In [36]:
#len(updatedChartEventsData)

In [37]:
updatedInputEventsData2 = updatedInputEventsData.drop(columns=['starttime', 'endtime'])

In [38]:
updatedInputEventsData2['storetime'].head()

0    2180-07-23 17:02:00
1    2180-07-23 18:16:00
2    2156-04-21 11:48:00
3    2156-04-23 02:07:00
4    2156-04-23 05:19:00
Name: storetime, dtype: object

In [39]:
updatedInputEventsData2['storetime'] = dd.to_datetime(updatedInputEventsData2['storetime'])
updatedChartEventsData['storetime'] = dd.to_datetime(updatedChartEventsData['storetime'])

In [40]:
updatedInputEventsData2_sorted = [part.sort_values(by='storetime') for part in updatedInputEventsData2.to_delayed()]
updatedInputEventsData2_sorted = dd.from_delayed(updatedInputEventsData2_sorted, meta=updatedInputEventsData2)

In [41]:
updatedInputEventsData2_sorted.head()

,itemid,label,category,subject_id,hadm_id,stay_id,storetime,amount,amountuom,rate,rateuom,ordercategoryname,secondaryordercategoryname
118352,225158,NaCl 0.9%,Fluids/Intake,10129284.0,27250521.0,36352880.0,2110-02-16 23:20:00,117.666670,ml,10.000000,mL/hour,02-Fluids (Crystalloids),Additive (Crystalloid)
101680,225153,Labetalol,Medications,10129284.0,27250521.0,36352880.0,2110-02-16 23:49:00,10.000001,mg,NaN,NaN,05-Med Bolus,NaN
217853,226452,PO Intake,Fluids/Intake,10129284.0,27250521.0,36352880.0,2110-02-17 04:38:00,360.000000,ml,NaN,NaN,14-Oral/Gastric Intake,NaN
217854,226452,PO Intake,Fluids/Intake,10129284.0,27250521.0,36352880.0,2110-02-17 09:27:00,240.000000,ml,NaN,NaN,14-Oral/Gastric Intake,NaN
100074,225152,Heparin Sodium,Medications,10098627.0,20711347.0,39342860.0,2110-03-20 18:24:00,17035.147421,units,850.340149,units/hour,01-Drips,02-Fluids (Crystalloids)


In [42]:
#updatedInputEventsData2_sorted = updatedInputEventsData2.sort_values(by='storetime').compute()

In [43]:
updatedChartEventsData.head()

,itemid,label,category,subject_id,hadm_id,stay_id,charttime,storetime,value,valuenum,valueuom,warning
0,220001,Problem List,General,10000980.0,26913865.0,39765666.0,2189-06-27 15:24:57,NaT,.Care Plan - Altered Respiratory Function: Ine...,NaN,NaN,NaN
1,220001,Problem List,General,10001217.0,24597018.0,37067082.0,2157-11-21 07:26:41,NaT,.Care Plan - Post-operative,NaN,NaN,NaN
2,220001,Problem List,General,10001217.0,27703517.0,34592300.0,2157-12-19 21:56:58,NaT,.Care Plan - Post-operative,NaN,NaN,NaN
3,220001,Problem List,General,10001725.0,25563031.0,31205490.0,2110-04-11 18:56:18,NaT,.Care Plan - Post-operative,NaN,NaN,NaN
4,220001,Problem List,General,10001725.0,25563031.0,31205490.0,2110-04-11 20:39:56,NaT,.Care Plan - Altered Respiratory Function: Ine...,NaN,NaN,NaN


In [44]:
updatedChartEventsData_sorted = [part.sort_values(by='storetime') for part in updatedChartEventsData.to_delayed()]
updatedChartEventsData_sorted = dd.from_delayed(updatedChartEventsData_sorted, meta=updatedChartEventsData)


In [45]:
updatedChartEventsData_sorted.head()

,itemid,label,category,subject_id,hadm_id,stay_id,charttime,storetime,value,valuenum,valueuom,warning
120706,223758,Code Status,General,10001725.0,25563031.0,31205490.0,2110-04-11 16:01:00,2110-04-11 16:01:00,Full code,NaN,NaN,0.0
348704,224640,Service,General,10001725.0,25563031.0,31205490.0,2110-04-11 16:01:00,2110-04-11 16:01:00,MICU/SICU,NaN,NaN,0.0
471943,226512,Admission Weight (Kg),General,10001725.0,25563031.0,31205490.0,2110-04-11 15:52:00,2110-04-11 16:01:00,72.2,72.2,kg,0.0
470220,226253,SpO2 Desat Limit,Alarms,10001725.0,25563031.0,31205490.0,2110-04-11 16:00:00,2110-04-11 16:01:00,85,85.0,%,0.0
323853,224162,Resp Alarm - Low,Alarms,10001725.0,25563031.0,31205490.0,2110-04-11 16:00:00,2110-04-11 16:01:00,8,8.0,insp/min,0.0


In [46]:
# updatedChartEventsData_sorted = updatedChartEventsData.sort_values(by='storetime').compute()

In [47]:
#updatedInputEventsData2_sorted.dropna(subset=['storetime'], inplace=True)
#updatedChartEventsData_sorted.dropna(subset=['storetime'], inplace=True)

In [48]:
batch_size = 10

# Perform merge in smaller batches
merged_batches = []


# Perform merge in smaller batches
for i in range(0, updatedInputEventsData2_sorted.npartitions, batch_size):
    updatedInputEventsData2_sorted_batch = updatedInputEventsData2_sorted.get_partition(i)
    updatedChartEventsData_sorted_batch = updatedChartEventsData_sorted.get_partition(i)
    merged_batch = dd.merge(updatedInputEventsData2_sorted_batch, updatedChartEventsData_sorted_batch, on='storetime')
    merged_batches.append(merged_batch)

# Concatenate the results
merged_result = dd.concat(merged_batches)

# Compute the final result (optional, depending on your further operations)
merged_result = merged_result.compute()

# Display the result
print(merged_result)

      itemid_x              label_x     category_x  subject_id_x   hadm_id_x  \
0       229072  Ketorolac (Toradol)    Medications    10001725.0  25563031.0   
1       229072  Ketorolac (Toradol)    Medications    10001725.0  25563031.0   
2       229072  Ketorolac (Toradol)    Medications    10001725.0  25563031.0   
3       229072  Ketorolac (Toradol)    Medications    10001725.0  25563031.0   
4       229072  Ketorolac (Toradol)    Medications    10001725.0  25563031.0   
...        ...                  ...            ...           ...         ...   
3831    225158            NaCl 0.9%  Fluids/Intake    18553859.0  24075018.0   
3832    225158            NaCl 0.9%  Fluids/Intake    18553859.0  24075018.0   
3833    225158            NaCl 0.9%  Fluids/Intake    18553859.0  24075018.0   
3834    225158            NaCl 0.9%  Fluids/Intake    18553859.0  24075018.0   
3835    225158            NaCl 0.9%  Fluids/Intake    18553859.0  24075018.0   

       stay_id_x           storetime   

In [49]:
# Assuming 'time' is the common time column
#merged_data = dd.merge_asof(updatedInputEventsData2_sorted, updatedChartEventsData, on='storetime')
merged_result.head()

,itemid_x,label_x,category_x,subject_id_x,hadm_id_x,stay_id_x,storetime,amount,amountuom,rate,...,label_y,category_y,subject_id_y,hadm_id_y,stay_id_y,charttime,value,valuenum,valueuom,warning
0,229072,Ketorolac (Toradol),Medications,10001725.0,25563031.0,31205490.0,2110-04-11 22:52:00,15.000001,mg,NaN,...,Pain Type,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,Constant,NaN,NaN,0.0
1,229072,Ketorolac (Toradol),Medications,10001725.0,25563031.0,31205490.0,2110-04-11 22:52:00,15.000001,mg,NaN,...,Pain Level Acceptable,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,Tolerable,NaN,NaN,0.0
2,229072,Ketorolac (Toradol),Medications,10001725.0,25563031.0,31205490.0,2110-04-11 22:52:00,15.000001,mg,NaN,...,Pain Cause,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,At Rest,NaN,NaN,0.0
3,229072,Ketorolac (Toradol),Medications,10001725.0,25563031.0,31205490.0,2110-04-11 22:52:00,15.000001,mg,NaN,...,Pain Level Response,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,None,0.0,NaN,0.0
4,229072,Ketorolac (Toradol),Medications,10001725.0,25563031.0,31205490.0,2110-04-11 22:52:00,15.000001,mg,NaN,...,Untoward Effect,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,None,NaN,NaN,0.0


In [50]:
filtered_df = merged_result.query("subject_id_x == subject_id_y and hadm_id_x==hadm_id_y and stay_id_x==stay_id_y")

In [51]:
filtered_df = filtered_df.drop(columns = ['subject_id_x', 'hadm_id_x', 'stay_id_x'])

In [52]:
filtered_df = filtered_df.rename(columns = {'subject_id_y':'subject_id', 'hadm_id_y': 'hadm_id', 'stay_id_y':'stay_id'})

In [53]:
filtered_df

,itemid_x,label_x,category_x,storetime,amount,amountuom,rate,rateuom,ordercategoryname,secondaryordercategoryname,...,label_y,category_y,subject_id,hadm_id,stay_id,charttime,value,valuenum,valueuom,warning
0,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Pain Type,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,Constant,NaN,NaN,0.0
1,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Pain Level Acceptable,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,Tolerable,NaN,NaN,0.0
2,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Pain Cause,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,At Rest,NaN,NaN,0.0
3,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Pain Level Response,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,None,0.0,NaN,0.0
4,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Untoward Effect,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,None,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60313,225798,Vancomycin,Antibiotics,2200-10-05 00:00:00,1.000000,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids),...,Parameters Checked,Alarms,10013643.0,22009484.0,33072499.0,2200-10-05 00:00:00,1,1.0,NaN,0.0
60314,225798,Vancomycin,Antibiotics,2200-10-05 00:00:00,1.000000,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids),...,Pain Present,Pain/Sedation,10013643.0,22009484.0,33072499.0,2200-10-05 00:00:00,No,NaN,NaN,0.0
60315,225798,Vancomycin,Antibiotics,2200-10-05 00:00:00,1.000000,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids),...,Temperature Site,Routine Vital Signs,10013643.0,22009484.0,33072499.0,2200-10-05 00:00:00,Oral,NaN,NaN,0.0
60316,225798,Vancomycin,Antibiotics,2200-10-05 00:00:00,1.000000,dose,NaN,NaN,08-Antibiotics (IV),02-Fluids (Crystalloids),...,ST Segment Monitoring On,Alarms,10013643.0,22009484.0,33072499.0,2200-10-05 00:00:00,1,1.0,NaN,0.0


In [54]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58095 entries, 0 to 60317
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   itemid_x                    58095 non-null  int64         
 1   label_x                     58095 non-null  object        
 2   category_x                  58095 non-null  object        
 3   storetime                   58095 non-null  datetime64[ns]
 4   amount                      58095 non-null  float64       
 5   amountuom                   58095 non-null  object        
 6   rate                        29176 non-null  float64       
 7   rateuom                     29176 non-null  object        
 8   ordercategoryname           58095 non-null  object        
 9   secondaryordercategoryname  38687 non-null  object        
 10  itemid_y                    58095 non-null  int64         
 11  label_y                     58095 non-null  object    

In [55]:
filtered_df.size

1219995

In [56]:
filtered_df['itemid_y'].unique

<bound method Series.unique of 0        223782
1        223794
2        223784
3        224409
4        223796
          ...  
60313    224168
60314    223781
60315    224642
60316    228305
60317    224641
Name: itemid_y, Length: 58095, dtype: int64>

In [57]:
from sklearn.preprocessing import OneHotEncoder

In [58]:
def oneHotEncoder(columnName, df, newColumnNames):
  #creating instance of one-hot-encoder
  encoder = OneHotEncoder(handle_unknown='ignore')
  #perform one-hot encoding on 'team' column
  encoder_df = pd.DataFrame(encoder.fit_transform(df[[columnName]]).toarray())
  #print(encoder_df)
  #merge one-hot encoded columns back with original DataFrame
  final_df = df.join(encoder_df)
  final_df = final_df.drop(columns=columnName)
  columnRenaming={}
  for i in range(len(newColumnNames)):
    columnRenaming[i]=newColumnNames[i]
  #print(columnRenaming)
  #final_df = final_df.rename(columns = columnRenaming)
  return final_df

In [59]:
filtered_df.size

1219995

In [60]:
grouped_df = filtered_df.groupby('itemid_y')

In [62]:
grouped_df.head()

,itemid_x,label_x,category_x,storetime,amount,amountuom,rate,rateuom,ordercategoryname,secondaryordercategoryname,...,label_y,category_y,subject_id,hadm_id,stay_id,charttime,value,valuenum,valueuom,warning
0,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Pain Type,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,Constant,NaN,NaN,0.0
1,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Pain Level Acceptable,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,Tolerable,NaN,NaN,0.0
2,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Pain Cause,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,At Rest,NaN,NaN,0.0
3,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Pain Level Response,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,None,0.0,NaN,0.0
4,229072,Ketorolac (Toradol),Medications,2110-04-11 22:52:00,15.000001,mg,NaN,NaN,05-Med Bolus,NaN,...,Untoward Effect,Pain/Sedation,10001725.0,25563031.0,31205490.0,2110-04-11 22:00:00,None,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59053,220949,Dextrose 5%,Fluids/Intake,2196-02-27 03:25:00,176.225009,ml,40.051140,mL/hour,01-Drips,02-Fluids (Crystalloids),...,Arctic Sun Temp #2 Location,Routine Vital Signs,10004235.0,24181354.0,34100191.0,2196-02-27 03:00:00,Rectal,NaN,NaN,0.0
59928,221749,Phenylephrine,Medications,2200-10-02 02:25:00,21.115480,mg,1.397081,mcg/kg/min,01-Drips,02-Fluids (Crystalloids),...,Knee Location - Immobilizer,Treatments,10013643.0,22009484.0,33072499.0,2200-10-02 00:00:00,Both Knees,NaN,NaN,0.0
59929,221749,Phenylephrine,Medications,2200-10-02 02:25:00,21.115480,mg,1.397081,mcg/kg/min,01-Drips,02-Fluids (Crystalloids),...,Knee Immobilizer Status,Treatments,10013643.0,22009484.0,33072499.0,2200-10-02 00:00:00,On,NaN,NaN,0.0
59932,220949,Dextrose 5%,Fluids/Intake,2200-10-02 02:25:00,87.981187,ml,23.051840,mL/hour,01-Drips,02-Fluids (Crystalloids),...,Knee Location - Immobilizer,Treatments,10013643.0,22009484.0,33072499.0,2200-10-02 00:00:00,Both Knees,NaN,NaN,0.0


In [63]:
group_a_df = grouped_df.get_group(229247).reset_index(drop=True)
print(group_a_df.head())
# Now group_a_df is a regular DataFrame containing only the 'A' group
print(group_a_df.info())

   itemid_x                 label_x     category_x           storetime  \
0    225158               NaCl 0.9%  Fluids/Intake 2136-01-26 03:06:00   
1    221906          Norepinephrine    Medications 2136-01-26 03:06:00   
2    227526                 Citrate    Medications 2136-01-26 07:21:00   
3    227529  ACD-A Citrate (1000ml)    Medications 2136-01-26 07:21:00   
4    221906          Norepinephrine    Medications 2136-01-26 09:04:00   

       amount amountuom        rate     rateuom         ordercategoryname  \
0   11.331038        ml    7.156445     mL/hour                  01-Drips   
1    0.362593        mg    0.059918  mcg/kg/min                  01-Drips   
2  110.303308      mmol         NaN         NaN  02-Fluids (Crystalloids)   
3  976.999999        ml  200.068253     mL/hour  02-Fluids (Crystalloids)   
4    0.391698        mg    0.109805  mcg/kg/min                  01-Drips   

  secondaryordercategoryname  ...                  label_y category_y  \
0   02-Fluids (Crys

In [64]:
from sklearn.model_selection import train_test_split
def split_dataset(X, y, split_test_size = 0.2, shuffle = False):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_test_size, random_state=SEED, shuffle=shuffle)

  return [X_train, X_test, y_train, y_test]

In [65]:
[X_train, X_test, y_train, y_test] = split_dataset(group_a_df['valuenum'], group_a_df['amount'])

In [67]:
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)

In [73]:
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X_train, np.array(y_train))

y_pred = reg.predict(X_test)

print(reg.score(X_test, y_test))

print("MSE = ", metrics.mean_absolute_error(y_test, y_pred))
print("MAE = ", metrics.mean_squared_error(y_test, y_pred))
print("RMSE = ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


-0.03932950393643608
MSE =  177.44660262443773
MAE =  90628.3090785396
RMSE =  301.0453604999413


In [ ]:
!pip install tensorflow

In [74]:
grouped_df_patients = filtered_df.groupby('subject_id')

In [75]:
grouped_df_patients

In [76]:
# Displaying the groups
"""
for group_name, group_data in grouped_df_patients:
    print(f"Group {group_name}:")
    print(group_data['amount'])
    print(group_data['valuenum'])
    print("\n")
"""

'\nfor group_name, group_data in grouped_df_patients:\n    print(f"Group {group_name}:")\n    print(group_data[\'amount\'])\n    print(group_data[\'valuenum\'])\n    print("\n")\n'

In [77]:
#supervised learning model 2: Support Vector Regression
X_train = np.array(X_train).reshape(-1,1)
X_test = np.array(X_test).reshape(-1,1)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR

# create an SVR model with a linear kernel
svr = SVR(kernel='linear')

# train the model on the data
svr.fit(X_train, y_train)

# make predictions on the data
y_pred = svr.predict(X_test)

In [78]:
print("MSE = ", metrics.mean_absolute_error(y_test, y_pred))
print("MAE = ", metrics.mean_squared_error(y_test, y_pred))
print("RMSE = ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

MSE =  165.43718170566882
MAE =  105628.72323913923
RMSE =  325.00572800973714
